# Environment Setup

In [ ]:
#Pretty standard stuff here

!mkdir PongReinforcementLearning
!cd PongReinforcementLearning

# Then, I set up a virtual environment (venv)
python -m venv PongReinforcementLearningVENV
!source PongReinforcementLearningVENV/bin/activate

# Make the venv recognizable to Jupyter Notebooks.
# This is the bridge that connects Jupyter to my isolated Python environment.
%pip install ipyconfig
python -m ipykernel install --user --name=PongReinforcementLearningVENV

# Time to fire up Jupyter Notebook.
# Make sure to select the new venv as the Python interpreter.
jupyter notebook

# Finally, installing some libs, i usually do these via the console but Jupyter's % operator usually works just fine
%pip3 install pygame
%pip install numpy
%pip install pandas

# See if I can run an external Pygame window from a Jupyter notebook on macosx

In [2]:
import pygame
pygame.init()

# Create external window
win = pygame.display.set_mode((500, 500))

# Main game loop
run = True
while run:
    pygame.time.delay(100)
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False
            
    # Game logic here (e.g., move a rectangle)
    pygame.draw.rect(win, (255, 0, 0), (250, 250, 50, 50))
    
    pygame.display.update()

pygame.quit()


pygame 2.5.1 (SDL 2.28.2, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


**Well, it runs but shutdown isn't graceful.  The window pops up, draws a glorious red square.  But then simple window commands like "close" fail.  I had to Force Quit which then also brought the Jupyter notebook kernel to the ground.  This may wind up being a royal PITA but i'll give it a shot for now.  Worst case I'll switch to a simple python script run from the console.**

# Pong

In [ ]:
import pygame
import random
import numpy as np  
import pickle
import os
import math
import logging
import datetime

#Helper function to load data from a pickle file
def load_data_from_pickle_file(filename, default_value):
    try: return pickle.load(open(filename, "rb")) if os.path.exists(filename) else default_value
    except Exception as e: print(f"Error loading {filename}: {e}"); return default_value

#Convert input coordinate to discrete grid space.  this smaller grid space should make learning easier.
def discretize_grid(coordinate): 
    return coordinate // GAME_BOARD_GRID_SIZE

#Convert velocity into discretized space (of only 4 options!)
def discretize_velocity(velocity_x, velocity_y):
    if velocity_x > 0 and velocity_y > 0:
        return 0  # Up-Right
    elif velocity_x > 0 and velocity_y < 0:
        return 1  # Down-Right
    elif velocity_x < 0 and velocity_y > 0:
        return 2  # Up-Left
    elif velocity_x < 0 and velocity_y < 0:
        return 3  # Down-Left
    
#Main Pong game function, accepts key parameters as inputs now
def play_de_game(DATA_FILE_PREFIX, episodes_to_run, alpha, gamma, epsilon, epsilon_min, epsilon_decay, GAME_BOARD_GRID_SIZE, reward_for_winning_episode, punishment_for_losing_episode, reward_for_hitting_ball):
    
    #Key Results
    KR_reward_events_left = 0
    KR_reward_events_right = 0
    KR_ball_hits_left = 0
    KR_ball_hits_right = 0
    KR_avg_episode_length = 0
    KR_epsilon_values = []
    
    # Init files
    Q_TABLE_LEFT_FILE = 'data/' + DATA_FILE_PREFIX + 'Q_table_left.pkl'
    Q_TABLE_RIGHT_FILE = 'data/' + DATA_FILE_PREFIX + 'Q_table_right.pkl'
    EPISODE_COUNT_FILE = 'data/' + DATA_FILE_PREFIX + 'Episode_count.pkl'
    
    #Q-table save frequency
    episode_count = 0  # Initialize episode count
    save_frequency = 100  # Save every 100 episodes
    
    # Init Q-tables
    Q_table_left = {}
    Q_table_right = {}
    
    #Load data from pickle
    Q_table_left = load_data_from_pickle_file(Q_TABLE_RIGHT_FILE, {})
    Q_table_right = load_data_from_pickle_file(Q_TABLE_RIGHT_FILE, {})
    episode_count = load_data_from_pickle_file(EPISODE_COUNT_FILE, 0)
    
    # Initialize scores
    left_score = 0
    right_score = 0
    
    # Initial paddle positions
    #left_paddle_pos = [50, height // 2 - paddle_height // 2]
    #right_paddle_pos = [width - 50 - paddle_width, height // 2 - paddle_height // 2]
    # Paddle positions to a random spot
    left_paddle_pos = [50, random.randint(0, height - paddle_height)]
    right_paddle_pos = [width - 50 - paddle_width, random.randint(0, height - paddle_height)]
    ball_pos = [width // 2, height // 2]
    
    # Initial Ball velocity
    ball_velocity = [random.choice([-4, 4]), random.choice([-4, 4])]
    
    # Initialize this_episode_length
    episode_lengths = []
    this_episode_length = 0
    
    # Init whether each AI agent has hit the ball in this episode yet
    contact_with_ball_made_this_episode_left = False
    contact_with_ball_made_this_episode_right = False
    
    # Init results
    results = {}
    
    run = True
    user_quit = False
    while run:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                run = False
                user_quit = True

        #Track game loops in this episode/game and report to screen to get a sense of how many loops a game lasts
        this_episode_length += 1

        #Debug track whether we have a rewarded event in this loop
        reward_applied_this_loop = False

        # Reset rewards to 0 at the beginning of each pass through the game loop
        reward_left = 0
        reward_right = 0

        # Create the state representation for both agents
        state_left = (discretize_grid(left_paddle_pos[1]), discretize_grid(ball_pos[0]), discretize_grid(ball_pos[1]), discretize_velocity(ball_velocity[0], ball_velocity[1]))
        state_right = (discretize_grid(right_paddle_pos[1]), discretize_grid(ball_pos[0]), discretize_grid(ball_pos[1]), discretize_velocity(ball_velocity[0], ball_velocity[1]))

        # Initialize Q-values for the states if not already present
        if state_left not in Q_table_left:
            Q_table_left[state_left] = {action: np.random.uniform(-1, 1) for action in action_space}
        if state_right not in Q_table_right:
            Q_table_right[state_right] = {action: np.random.uniform(-1, 1) for action in action_space}

        # Choose an action for both agents using the epsilon-greedy policy
        action_left = max(Q_table_left[state_left], key=Q_table_left[state_left].get) if np.random.rand() >= epsilon else np.random.choice(action_space)
        action_right = max(Q_table_right[state_right], key=Q_table_right[state_right].get) if np.random.rand() >= epsilon else np.random.choice(action_space)

        # Manual human paddle movement with boundary checks
        #keys = pygame.key.get_pressed()
        #if keys[pygame.K_w] and left_paddle_pos[1] > 0:
        #    left_paddle_pos[1] -= 5
        #if keys[pygame.K_s] and left_paddle_pos[1] < height - paddle_height:
        #    left_paddle_pos[1] += 5
        #if keys[pygame.K_UP] and right_paddle_pos[1] > 0:
        #    right_paddle_pos[1] -= 5
        #if keys[pygame.K_DOWN] and right_paddle_pos[1] < height - paddle_height:
        #    right_paddle_pos[1] += 5

        #Left AI agent moves the paddle!!
        if action_left == 0 and left_paddle_pos[1] > 0:  # Move Up
            left_paddle_pos[1] -= 5
        elif action_left == 1 and left_paddle_pos[1] < height - paddle_height:  # Move Down
            left_paddle_pos[1] += 5
        #elif action_left == 2: 
            # Stay Still, so no movement

        #Right AI agent moves the paddle!!
        if action_right == 0 and right_paddle_pos[1] > 0:  # Move Up
            right_paddle_pos[1] -= 5
        elif action_right == 1 and right_paddle_pos[1] < height - paddle_height:  # Move Down
            right_paddle_pos[1] += 5
        #elif action_right == 2: 
            # Stay Still, so no movement

        # Debugging code to print current state and action for both agents
        #print(f"Current State Left: {state_left}, Action Taken Left: {action_left}")
        #print(f"Current State Right: {state_right}, Action Taken Right: {action_right}")

        # Update ball position
        ball_pos[0] += ball_velocity[0]
        ball_pos[1] += ball_velocity[1]

        # Collision detection with walls
        if ball_pos[1] <= 0 or ball_pos[1] >= height:
            ball_velocity[1] = -ball_velocity[1]

        # Collision detection with paddles
        collision_offset = 5  # Define an offset to push the ball away from the paddle
        if (left_paddle_pos[0] <= ball_pos[0] <= left_paddle_pos[0] + paddle_width and
            left_paddle_pos[1] <= ball_pos[1] <= left_paddle_pos[1] + paddle_height):
            ball_velocity[0] = -ball_velocity[0]
            ball_pos[0] += collision_offset  # Push the ball away from the paddle
            reward_left = reward_for_hitting_ball  # Add reward for left agent
            contact_with_ball_made_this_episode_left = True # Note that left agent has contacted ball this episode
            reward_applied_this_loop = True
            KR_ball_hits_left += 1 # Tracked to evaluate run of episodes
            KR_reward_events_left += 1 # Tracked to evaluate run of episodes
        elif (right_paddle_pos[0] <= ball_pos[0] <= right_paddle_pos[0] + paddle_width and
              right_paddle_pos[1] <= ball_pos[1] <= right_paddle_pos[1] + paddle_height):
            ball_velocity[0] = -ball_velocity[0]
            ball_pos[0] -= collision_offset  # Push the ball away from the paddle
            reward_right = reward_for_hitting_ball  # Add reward for right agent
            contact_with_ball_made_this_episode_right = True # Note that right agent has contacted ball this episode
            reward_applied_this_loop = True
            KR_ball_hits_right += 1 # Tracked to evaluate run of episodes
            KR_reward_events_right += 1 # Tracked to evaluate run of episodes

        #Penalties for not exploring enough
        #extreme_zones = [[0, height // 8], [7 * height // 8, height]]  # Define the extreme zones
        #bonus = 0.1  # Define the bonus
        #center_zone = [height // 4, 3 * height // 4]  # Define the center zone
        #penalty = -0.1  # Define the penalty
        # Apply penalty for center zone
        #if center_zone[0] <= left_paddle_pos[1] <= center_zone[1]:
        #    reward_left += penalty
        #if center_zone[0] <= right_paddle_pos[1] <= center_zone[1]:
        #    reward_right += penalty
        # Apply bonus for extreme zones
        #for zone in extreme_zones:
        #    if zone[0] <= left_paddle_pos[1] <= zone[1]:
        #        reward_left += bonus
        #    if zone[0] <= right_paddle_pos[1] <= zone[1]:
        #        reward_right += bonus

        # Ball reset, scoring, and immediate feedback game-over condition
        if ball_pos[0] < 0:
            # Reset paddle positions to the middle
            #left_paddle_pos = [50, height // 2 - paddle_height // 2]
            #right_paddle_pos = [width - 50 - paddle_width, height // 2 - paddle_height // 2]
            # Reset paddle positions to a random spot
            left_paddle_pos = [50, random.randint(0, height - paddle_height)]
            right_paddle_pos = [width - 50 - paddle_width, random.randint(0, height - paddle_height)]
            #Reset the ball to the center in a random direction
            ball_pos = [width // 2, height // 2]
            ball_velocity = [random.choice([-4, 4]), random.choice([-4, 4])]
            #Scoring
            right_score += 1  # Right player scores
            KR_reward_events_right += 1 # Tracked to evaluate run of episodes
            #Rewards
            reward_left += punishment_for_losing_episode  # Punishment for the left agent
            if contact_with_ball_made_this_episode_right: # Only reward if right agent made contact in this episode
                reward_right += reward_for_winning_episode  # Positive reward for the right agent
            reward_applied_this_loop = True
            contact_with_ball_made_this_episode_left = False # Reset
            contact_with_ball_made_this_episode_right = False # Reset
            #Signal the end of an episode
            episode_count += 1  # Increment episode count
            # Keep track of average episode length
            episode_lengths.append(this_episode_length)
            KR_avg_episode_length = sum(episode_lengths) / len(episode_lengths)
            this_episode_length = 0 # Reset length of episode
            # Decay epsilon at the end of a game/episode
            if epsilon > epsilon_min:
                epsilon *= epsilon_decay
            #KR_epsilon_values.append(epsilon) # Tracked to evaluate run of episodes
            # Save the Q-tables every save_frequency episodes
            #if episode_count % save_frequency == 0:
            #    with open(Q_TABLE_LEFT_FILE, "wb") as f:
            #        pickle.dump(Q_table_left, f)
            #    with open(Q_TABLE_RIGHT_FILE, "wb") as f:
            #        pickle.dump(Q_table_right, f)
            #    with open(EPISODE_COUNT_FILE, "wb") as f:
            #        pickle.dump(episode_count, f)
        elif ball_pos[0] > width:
            # Reset paddle positions to the middle
            #left_paddle_pos = [50, height // 2 - paddle_height // 2]
            #right_paddle_pos = [width - 50 - paddle_width, height // 2 - paddle_height // 2]
            # Reset paddle positions to a random spot
            left_paddle_pos = [50, random.randint(0, height - paddle_height)]
            right_paddle_pos = [width - 50 - paddle_width, random.randint(0, height - paddle_height)]
            #Reset the ball to the center in a random direction
            ball_pos = [width // 2, height // 2]
            ball_velocity = [random.choice([-4, 4]), random.choice([-4, 4])]
            #Scoring
            left_score += 1  # Left player scores
            KR_reward_events_left += 1 # Tracked to evaluate run of episodes
            #Rewards
            if contact_with_ball_made_this_episode_left: # Only reward if left agent made contact in this episode
                reward_left += reward_for_winning_episode  # Positive reward for the left agent
            reward_right += punishment_for_losing_episode  # Punishment for the right agent
            reward_applied_this_loop = True
            contact_with_ball_made_this_episode_left = False # Reset
            contact_with_ball_made_this_episode_right = False # Reset
            #Signal the end of an episode
            episode_count += 1  # Increment episode count
            # Keep track of average episode length
            episode_lengths.append(this_episode_length)
            KR_avg_episode_length = sum(episode_lengths) / len(episode_lengths)
            this_episode_length = 0 # Reset length of episode
            # Decay epsilon at the end of a game/episode
            if epsilon > epsilon_min:
                epsilon *= epsilon_decay
            #KR_epsilon_values.append(epsilon) # Tracked to evaluate run of episodes
            # Save the Q-tables every save_frequency episodes
            #if episode_count % save_frequency == 0:
            #    with open(Q_TABLE_LEFT_FILE, "wb") as f:
            #        pickle.dump(Q_table_left, f)
            #    with open(Q_TABLE_RIGHT_FILE, "wb") as f:
            #        pickle.dump(Q_table_right, f)
            #    with open(EPISODE_COUNT_FILE, "wb") as f:
            #        pickle.dump(episode_count, f)

        # After taking an action, observe new state and reward
        new_state_left = (discretize_grid(left_paddle_pos[1]), right_paddle_pos[1], discretize_grid(ball_pos[0]), discretize_grid(ball_pos[1]), discretize_velocity(ball_velocity[0], ball_velocity[1]))
        new_state_right = (discretize_grid(left_paddle_pos[1]), right_paddle_pos[1], discretize_grid(ball_pos[0]), discretize_grid(ball_pos[1]), discretize_velocity(ball_velocity[0], ball_velocity[1]))

        # Check if state has changed
        if new_state_left != state_left or new_state_right != state_right or reward_applied_this_loop:

            # Initialize Q-values-left for the new state if not already present
            if new_state_left not in Q_table_left:
                Q_table_left[new_state_left] = {action: 0 for action in action_space}

            # Initialize Q-values-right for the new state if not already present
            if new_state_right not in Q_table_right:
                Q_table_right[new_state_right] = {action: 0 for action in action_space}

            # Calculate the best next action for both agents
            best_next_action_left = max(Q_table_left[new_state_left], key=Q_table_left[new_state_left].get)
            best_next_action_right = max(Q_table_right[new_state_right], key=Q_table_right[new_state_right].get)

            # Q-Learning update rule for both agents
            Q_table_left[state_left][action_left] = (1 - alpha) * Q_table_left[state_left][action_left] + alpha * (reward_left + gamma * Q_table_left[new_state_left][best_next_action_left])
            Q_table_right[state_right][action_right] = (1 - alpha) * Q_table_right[state_right][action_right] + alpha * (reward_right + gamma * Q_table_right[new_state_right][best_next_action_right])

        # Update current state for next iteration
        state_left = new_state_left
        state_right = new_state_right

        # Draw paddles, ball, and scores
        #window.fill((0, 0, 0))  # Clear screen
        #pygame.draw.rect(window, (255, 255, 255), left_paddle_pos + [paddle_width, paddle_height])
        #pygame.draw.rect(window, (255, 255, 255), right_paddle_pos + [paddle_width, paddle_height])
        #pygame.draw.circle(window, (255, 255, 255), ball_pos, ball_radius)

        # Display scores
        #font = pygame.font.SysFont(None, 30)
        #score_display = font.render(f"score: {left_score} - {right_score}", True, (255, 255, 255))
        #window.blit(score_display, (width // 2 - 45, 10))

        # Display episode count
        #font = pygame.font.SysFont(None, 30)
        #episode_display = font.render(f"episodes played: {episode_count}", True, (255, 255, 255))
        #window.blit(episode_display, (width // 2 - 100, 40))

        # Display current epsilon
        #font = pygame.font.SysFont(None, 30)
        #epsilon_display = font.render(f"Epsilon: {epsilon:.4f}", True, (255, 255, 255))
        #window.blit(epsilon_display, (10, 70))

        # Display current frame within game
        #font = pygame.font.SysFont(None, 30)
        #epsilon_display = font.render(f"this_episode_length: {this_episode_length}", True, (255, 255, 255))
        #window.blit(epsilon_display, (10, 160))

        #pygame.display.update()

        if episode_count > episodes_to_run and episodes_to_run > 0:
            run = False
    
    results = {
        'config': {
            'alpha': alpha,
            'gamma': gamma,
            'epsilon': epsilon,
            'epsilon_min': epsilon_min,
            'epsilon_decay': epsilon_decay,
            'GAME_BOARD_GRID_SIZE': GAME_BOARD_GRID_SIZE,
            'reward_for_winning_episode': reward_for_winning_episode,
            'punishment_for_losing_episode': punishment_for_losing_episode,
            'reward_for_hitting_ball': reward_for_hitting_ball
        },
        'metrics': {
            'KR_reward_events_left': KR_reward_events_left,
            'KR_reward_events_right': KR_reward_events_right,
            'KR_ball_hits_left': KR_ball_hits_left,
            'KR_ball_hits_right': KR_ball_hits_right,
            'KR_avg_episode_length': KR_avg_episode_length
        }
    }
    
    return results, user_quit
 
    
#Constants
DATA_FILE_PREFIX = 'v1-'
DEBUG_OFF = 0
DEBUG_INFO = 1
DEBUG_DEBUG = 2
DEBUG_LEVEL = DEBUG_OFF # Default debug level setting
GAME_BOARD_GRID_SIZE = 50

#Rewards
reward_for_winning_episode = 1
reward_for_hitting_ball = 1
punishment_for_losing_episode = -1

# Initialize epsilon for the epsilon-greedy policy
epsilon = 1.0 #(orig 1.0)
epsilon_min = 0.10 #(orig .01)
epsilon_decay = 0.999995 #(orig .995)

# Initialize hyperparameters
alpha = 0.15  # Learning rate (orig .1)
gamma = 0.95  # Discount factor (orig .99)

# Define the action space
action_space = [0, 1, 2]  # 0: Move Up, 1: Move Down, 2: Stay Still

# Initialize Pygame
pygame.init()

# Create a window
width, height = 800, 600  # Window dimensions
window = pygame.display.set_mode((width, height))
pygame.display.set_caption('AI Learns Pong')

# Initialize paddle and ball attributes
paddle_width, paddle_height = 20, 100
ball_radius = 15

# Initialize logging
logging.basicConfig(level=logging.INFO)

# Define ranges and step sizes for key parameters
alpha_range = [0.1, 0.15, 0.2] # [0.1, 0.15, 0.2]
gamma_range = [0.9, 0.95, 0.99] # [0.9, 0.95, 0.99]
epsilon_range = [0.8, 0.9, 1.0] # [0.8, 0.9, 1.0]
epsilon_min_range = [0.05, 0.1, 0.15] # [0.05, 0.1, 0.15]
epsilon_decay_range = [0.995, 0.999, 0.9999] # [0.995, 0.999, 0.9999] 
reward_for_winning_episode_range = [1, 2] # [1, 2]
punishment_for_losing_episode_range = [-1, -2] # [-1, -2]
reward_for_hitting_ball_range = [0.5, 1, 1.5, 2.0] # [0.5, 1, 1.5]
GAME_BOARD_GRID_SIZE_range = [50, 100]

# For progress bar
total_runs = len(alpha_range) * len(gamma_range) * len(epsilon_range) * len(epsilon_min_range) * len(epsilon_decay_range) * len(reward_for_winning_episode_range) * len(punishment_for_losing_episode_range) * len(reward_for_hitting_ball_range) * len(GAME_BOARD_GRID_SIZE_range)
completed_runs = 0

# Initialize a dictionary to store all results
all_results = {}

# Flag to check if the user wants to quit
user_quit = False

# Nested loops to iterate through each parameter combination
for alpha in alpha_range:
    for gamma in gamma_range:
        for epsilon in epsilon_range:
            for epsilon_min in epsilon_min_range:
                for epsilon_decay in epsilon_decay_range:
                    for reward_for_winning_episode in reward_for_winning_episode_range:
                        for punishment_for_losing_episode in punishment_for_losing_episode_range:
                            for reward_for_hitting_ball in reward_for_hitting_ball_range:
                                for GAME_BOARD_GRID_SIZE in GAME_BOARD_GRID_SIZE_range:
                                    
                                    if user_quit:
                                        break

                                    # Generate a unique identifier for this parameter combination
                                    param_id = f"alpha-{alpha}_gamma-{gamma}_epsilon-{epsilon}_epsilon_min-{epsilon_min}_epsilon_decay-{epsilon_decay}_reward_win-{reward_for_winning_episode}_punish_lose-{punishment_for_losing_episode}_reward_hit-{reward_for_hitting_ball}"

                                    #logging.info(f"Running test for parameter set: {param_id}")
                                    
                                    start_time = datetime.datetime.now()

                                    # Run the game with the current parameter combination
                                    DATA_FILE_PREFIX_WITH_PARAMS = f"ParamTest-{DATA_FILE_PREFIX}{param_id}-"
                                    episodes_to_run = 2000
                                    results, user_quit = play_de_game(DATA_FILE_PREFIX_WITH_PARAMS, episodes_to_run, alpha, gamma, epsilon, epsilon_min, epsilon_decay, GAME_BOARD_GRID_SIZE, reward_for_winning_episode, punishment_for_losing_episode, reward_for_hitting_ball)

                                    # Store the results in the all_results dictionary
                                    all_results[param_id] = results
                                    #logging.info(f"Completed test for parameter set: {param_id}")
                                    
                                    completed_runs += 1
                                    percent_complete = (completed_runs / total_runs) * 100
                                    end_time = datetime.datetime.now()
                                    time_taken = end_time - start_time
                                    logging.info(f"Progress: {percent_complete:.2f}% Time taken: {time_taken} for {param_id[:20]}")

                                if user_quit:
                                    break
                            if user_quit:
                                break
                        if user_quit:
                            break
                    if user_quit:
                        break
                if user_quit:
                    break
            if user_quit:
                break
        if user_quit:
            break
    if user_quit:
        break

# Save all_results to a pickle file
if not user_quit:
    with open("data/"+DATA_FILE_PREFIX+"ParamTest-All-Results.pkl", "wb") as f:
        pickle.dump(all_results, f)

logging.info("All parameter tuning runs completed.")
        
pygame.quit()


# Parameter Tuning and Analysis for Reinforcement Learning in Pong

## Refactoring the Code

Initially, the Pong game was implemented in a monolithic script. To make it more modular and facilitate parameter tuning, the following steps were taken:

1. **Encapsulation**: The core game logic was encapsulated into a function called `play_de_game()`.
2. **Parameterization**: The function was designed to accept various hyperparameters as arguments, allowing for easy tuning. These parameters include:
    - `alpha`: The learning rate
    - `gamma`: The discount factor
    - `epsilon`: The exploration rate
    - `epsilon_min`: The minimum exploration rate
    - `epsilon_decay`: The decay rate for `epsilon`
    - `GAME_BOARD_GRID_SIZE`: The size of the game board grid
    - `reward_for_winning_episode`: The reward for winning an episode
    - `punishment_for_losing_episode`: The punishment for losing an episode
    - `reward_for_hitting_ball`: The reward for hitting the ball

## Running the Tests

After refactoring, the game was set up to run over a thousand tests, each with 2500 episodes. The tests were designed to explore a wide range of hyperparameters:

- **Alpha**: Learning rate, affecting how quickly the agent adapts to new information.
- **Gamma**: Discount factor, influencing how much future rewards are considered.
- **Epsilon**: Exploration rate, determining the likelihood of taking a random action.
- **Epsilon Min**: The minimum value that `epsilon` can decay to.
- **Epsilon Decay**: The rate at which `epsilon` decays over time.
- **Game Board Grid Size**: Affects the complexity of the state space.
- **Reward for Winning Episode**: Encourages the agent to win.
- **Punishment for Losing Episode**: Discourages the agent from losing.
- **Reward for Hitting Ball**: Encourages the agent to hit the ball.

The results of each test run were stored in a Python dictionary and then serialized to a pickle file (`all_results.pkl`) for later analysis.

## Data Analysis Plan

### Steps Involved:

1. **Load Data**: Import the `all_results.pkl` file into a Pandas DataFrame.
2. **Data Cleaning**: Remove any missing values and outliers, and convert columns to appropriate data types.
3. **Exploratory Data Analysis (EDA)**: Use statistical and visual methods to understand the data's underlying structure.
4. **Performance Metrics**: Evaluate the performance of different parameter sets based on metrics like average reward, episodes to convergence, etc.

### Expected Insights:

- **Optimal Parameters**: Identify the set of parameters that yield the best performance.
- **Parameter Sensitivity**: Understand how sensitive the model's performance is to changes in individual parameters.
- **Convergence Behavior**: Analyze how quickly the agent learns optimal policies under different parameter settings.
- **Reward Dynamics**: Examine how different reward structures affect the agent's learning process.

By the end of this analysis, I expect to have a comprehensive understanding of how different hyperparameters affect the learning process and performance of the reinforcement learning agent in the Pong game.


In [ ]:
import pandas as pd
import pickle

# Load the data from the pickle file
with open("data/v01-ParamTest-All-Results.pkl", "rb") as f:
    all_results = pickle.load(f)

# Initialize empty lists to store config and metrics data
config_data = []
metrics_data = []

# Iterate through the all_results dictionary to separate config and metrics
for param_id, result in all_results.items():
    config = result['config']
    metrics = result['metrics']
    
    # Add a parameter ID to link config and metrics
    config['param_id'] = param_id
    metrics['param_id'] = param_id
    
    config_data.append(config)
    metrics_data.append(metrics)

# Convert lists of dictionaries to DataFrames
config_df = pd.DataFrame(config_data)
metrics_df = pd.DataFrame(metrics_data)

# Merge the config and metrics DataFrames on 'param_id'
final_df = pd.merge(config_df, metrics_df, on='param_id')

# Data cleaning: Convert columns to appropriate data types, handle missing values, etc.
# For this example, I'm assuming all columns are of the correct data type and no missing values.
# You can add your own data cleaning steps here as needed.

# Display the first few rows of the final DataFrame
print(final_df.head())



# Notes

## Implementing Game Mechanics for Pong

### 1. Initialize Pygame and Create Window
- Initialized Pygame and created an 800x600 window for the game.

### 2. Initialize Paddle and Ball Attributes
- Defined the dimensions of the paddles and the ball. Initialized their starting positions.

### 3. Paddle Movement
- Implemented keyboard controls for moving the paddles up and down.

### 4. Ball Movement and Collision Detection
- Added logic for ball movement and collision detection with the walls and paddles.

### 5. Ball Reset and Scoring
- Implemented ball reset and scoring mechanics. The ball resets to the center after a point is scored.

### 6. Paddle Boundaries
- Added boundaries to prevent the paddles from moving out of the window.

### 7. Game Over Conditions
- Implemented immediate feedback game-over conditions. The game resets after each point, serving as an episode in RL terms.


## Defining RL Elements for Pong

### 1. State Representation
- Decide how to represent the state of the game. Consider the trade-offs between granularity and computational complexity.

### 2. Action Space
- Define the set of actions I can take (e.g., move paddle up, move paddle down, stay still).

### 3. Reward Structure
- Design the rewards I receive for various outcomes (e.g., +1 for scoring, -1 for opponent scoring).

### 4. Policy Initialization
- Initialize my policy, which could be a Q-table, a neural network, or some other function mapping states to actions.

### 5. Learning Algorithm
- Choose and implement a learning algorithm (e.g., Q-learning, SARSA, Deep Q-Networks) to update my policy based on experiences.

### 6. Exploration-Exploitation Strategy
- Decide on a strategy for balancing exploration (trying new actions) and exploitation (sticking with known good actions), such as ε-greedy.

### 7. Training Loop
- Implement the training loop where I interact with the environment, update my policy, and optionally log metrics like average reward over time.

### 8. Evaluation Metrics
- Define metrics to evaluate my performance (e.g., average reward, win rate).

### 9. Hyperparameter Tuning
- Experiment with different learning rates, discount factors, and other hyperparameters to optimize performance.

### 10. Testing and Validation
- Test the trained agent to see how well it performs and validate that it is learning effectively.


## Q-Learning Algorithm

Q-Learning is a model-free reinforcement learning algorithm that aims to learn a policy, which tells an agent what action to take under what circumstances. It defines a function \( Q(s, a) \), representing the quality or the utility of taking action \( a \) in state \( s \).

### Outline

1. **Initialize Q-Table**: Create a table to store the Q-values for each state-action pair.
2. **Policy**: Define how the agent chooses an action (e.g., \(\epsilon\)-greedy).
3. **Learning**: Update the Q-values using the Q-Learning update rule.
4. **Training Loop**: Incorporate these elements into the game loop.

The Q-table will be represented as a Python dictionary. The keys will be the states, and the values will be another dictionary mapping actions to Q-values.


## max() reference

| Iterable Type | What It Returns to `max()` | Example of Using `max()` |
|---------------|----------------------------|--------------------------|
| List          | Individual list elements   | `max([1, 2, 3])` returns `3` |
| Tuple         | Individual tuple elements  | `max((1, 2, 3))` returns `3` |
| String        | Individual characters     | `max("abc")` returns `'c'` |
| Set           | Individual set elements    | `max({1, 2, 3})` returns `3` |
| Dictionary    | Dictionary keys           | `max({'a': 1, 'b': 2}, key=lambda k: k)` returns `'b'` |
|               |                            | `max({'a': 1, 'b': 2}.values())` returns `2` |
|               |                            | `max({'a': 1, 'b': 2}, key=lambda k: {'a': 1, 'b': 2}[k])` returns `'b'` |
| Numpy Array   | Individual array elements  | `import numpy as np; max(np.array([1, 2, 3]))` returns `3` |


## Building intuition around training variables

1. **Alpha (α) - Learning Rate**: 
    - **What it does**: Determines how much of the new Q-value estimate I adopt.
    - **Intuition**: Think of it as a "blending factor." If α is 1, I consider only the most recent information. If α is 0, I learn nothing and stick to my prior knowledge. A value between 0 and 1 blends the old and new information.
    - **Example**: If α is high (closer to 1), I will rapidly adapt to new strategies but may also forget useful past knowledge quickly.

2. **Gamma (γ) - Discount Factor**: 
    - **What it does**: Influences how much future rewards contribute to the Q-value.
    - **Intuition**: It's like a "patience meter." A high γ makes me prioritize long-term reward over short-term reward.
    - **Example**: If γ is close to 1, I will consider future rewards with greater weight, making me more strategic but potentially slower to train.

3. **Epsilon (ε) - Exploration Rate**: 
    - **What it does**: Controls the trade-off between exploration (trying new actions) and exploitation (sticking with known actions).
    - **Intuition**: It's like the "curiosity level." A high ε encourages me to try new things, while a low ε makes me stick to what I know.
    - **Example**: If ε starts high and decays over time (ε-decay), I will initially explore a lot and gradually shift to exploiting my learned knowledge.
